<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_22_CrewAI_Home_Inspection_Email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to use Graphlit with CrewAI. Based on this [great project](https://github.com/bhancockio/crewai-rag-deep-dive/blob/main/README.md) written by Brandon Hancock, we emulate Brandon's workflow and show how to use Graphlit with CrewAI. In this example, we show the new IngestTool, which can be used to ingest files by URL into Graphlit, and the RetrievalTool, which can be used to search within a piece or content, or across your entire knowledge base.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.

For CrewAI, you will need your OpenAI API key.

Assign this property as Colab secret: OPENAI_API_KEY.

---

In [1]:
!pip install --upgrade graphlit-tools[crewai]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.9/211.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 82.5 MB/s eta

Initialize Graphlit

In [2]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [3]:
# NOTE: these functions are just used to clean-up old data before executing the example
async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)

Execute Graphlit sample

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
# Remove any existing contents; only needed for notebook example
await delete_all_contents()

print('Deleted all contents.')

Deleted all contents.


Initialize CrewAI

In [6]:
from crewai import Agent, Task, Crew, Process
from graphlit_tools import URLIngestTool, ContentRetrievalTool, CrewAIConverter
from textwrap import dedent

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# NOTE: can compare results using vector vs hybrid retrieval
#search_type = enums.SearchTypes.VECTOR
search_type = enums.SearchTypes.HYBRID

ingest_tool = CrewAIConverter.from_tool(URLIngestTool(graphlit))
retrieval_tool = CrewAIConverter.from_tool(ContentRetrievalTool(graphlit, search_type))

# --- Agents ---
research_agent = Agent(
    role="Research Agent",
    goal="Search through the PDF to find relevant answers",
    allow_delegation=False,
    verbose=True,
    backstory=dedent(
        """
        The research agent is adept at searching and
        extracting data from documents, ensuring accurate and prompt responses.
        """
    ),
    tools=[ingest_tool,retrieval_tool]
)

professional_writer_agent = Agent(
    role="Professional Writer",
    goal="Write professional emails based on the research agent's findings",
    allow_delegation=False,
    verbose=True,
    backstory=dedent(
        """
        The professional writer agent has excellent writing skills and is able to craft
        clear and concise emails based on the provided information.
        """
    )
)

# --- Tasks ---
answer_customer_question_task = Task(
    description=dedent(
        """
        Answer the customer's questions based on the home inspection PDF.
        The research agent will search through the PDF to find the relevant answers.
        Your final answer MUST be clear and accurate, based on the content of the home
        inspection PDF.

        Don't refence any source documents or add page numbers in your response.

        Here is the customer's question:
        {customer_question}

        You can access the PDF here: {pdf_url}
        """
    ),
    expected_output=dedent(
        """
        Provide clear and accurate answers to the customer's questions based on
        the content of the home inspection PDF.
        """
    ),
    agent=research_agent
)

write_email_task = Task(
    description=(
        """
        - Write a professional email to a contractor based
            on the research agent's findings.
        - The email should clearly state the issues found in the specified section
            of the report and request a quote or action plan for fixing these issues.
        - Ensure the email is signed with the following details:

            Best regards,

            John Doe,
            Generic Realty
        """
    ),
    expected_output="""
        Write a clear and concise email that can be sent to a contractor to address the
        issues found in the home inspection report.
        """,
    agent=professional_writer_agent,
)

# --- Crew ---
crew = Crew(
    agents=[research_agent, professional_writer_agent],
    tasks=[answer_customer_question_task, write_email_task],
    process=Process.sequential,
    verbose=True,
    planning=True
)

customer_question = input(
    "Which section of the report would you like to generate a work order for? (ex: Plumbing)\n"
)

pdf_url = "https://www.gpinspect.com/wp-content/uploads/2021/03/sample-home-report-inspection.pdf"

result = await crew.kickoff_async(inputs={"customer_question": customer_question, "pdf_url": pdf_url})

print(result)

Which section of the report would you like to generate a work order for? (ex: Plumbing)
Plumbing
 
[2025-01-15 04:24:00][INFO]: Planning the crew execution
# Agent: Research Agent
## Task: 
Answer the customer's questions based on the home inspection PDF.
The research agent will search through the PDF to find the relevant answers.
Your final answer MUST be clear and accurate, based on the content of the home
inspection PDF.

Don't refence any source documents or add page numbers in your response.

Here is the customer's question:
Plumbing

You can access the PDF here: https://www.gpinspect.com/wp-content/uploads/2021/03/sample-home-report-inspection.pdf
1. Use the Graphlit URL ingest tool to ingest the content of the home inspection PDF by entering the provided URL. This tool will parse the PDF and extract relevant information. 2. Once the content is ingested, utilize the Graphlit content retrieval tool to search for the term 'Plumbing' to locate all relevant sections that address plum